In [ ]:
%pip install --upgrade -r requirements.txt

Basic OpenAI Integration

In [1]:
from openai import OpenAI
import json
import random
import string

openai_key = "YOUR_OPENAI_KEY"
payi_api_key="YOUR_PAYI_API_KEY"

#Forward the OpenAI Authorization
payi_oai_headers = {
    "xProxy-Forward-Authorization": "Bearer " + openai_key
}

payi_base_url = "https://api.pay-i.com"
payi_oai_url = payi_base_url + "/api/v1/proxy/openai/v1"

oai_client = OpenAI(
    api_key=payi_api_key,
    base_url=payi_oai_url,
    default_headers=payi_oai_headers
)

response = oai_client.chat.completions.create(
    model="gpt-3.5-turbo", 
    messages=[{"role": "user", "content": "Say 'this is a test'"}],
)

completion = response.choices[0].message.content
print(completion)

xproxy_result = response.xproxy_result
print(json.dumps(xproxy_result, indent=4))

This is a test.
{
    "cost": {
        "currency": "usd",
        "input": {
            "base": 7e-06
        },
        "output": {
            "base": 7.5e-06
        },
        "total": {
            "base": 1.45e-05
        }
    },
    "request_id": "40000b84-0000-fe00-b63f-84710c7967bb"
}


Handle streaming calls. xproxy_result is returned as part of the last chunk.

In [2]:
stream = oai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "tell me a very short story"}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")
    if 'xproxy_result' in chunk.model_extra:
        print()
        print(json.dumps(chunk.model_extra['xproxy_result'], indent=4))

Once upon a time, in a small village nestled in the mountains, there lived a young girl named Lily. She was known for her kind heart and infectious laughter that could brighten even the gloomiest of days. One day, while exploring the forest near her home, Lily stumbled upon a wounded bird with a broken wing. Without hesitation, she gently picked up the bird and nursed it back to health with love and care. The bird, grateful for Lily's kindness, soon healed and flew off into the sky, leaving behind a trail of glittering feathers as a token of gratitude. From that day on, the villagers believed that Lily possessed a special connection with nature, and she was hailed as a guardian of the wilderness. Lily's act of kindness had not only helped a wounded creature but also brought harmony and joy to her village. And so, the legend of Lily the nature guardian spread far and wide, inspiring others to show compassion and care for all living beings.
{
    "cost": {
        "currency": "usd",
    

Use the Pay-i SDK to send a request with request tags

In [3]:
from payi.lib.helpers import create_headers

response = oai_client.chat.completions.create(
    model="gpt-3.5-turbo", 
    messages=[{"role": "user", "content": "Say 'this is a request tag test'"}],
    extra_headers=create_headers(request_tags=["x", "y"])
)

completion = response.choices[0].message.content
print(completion)

xproxy_result = response.xproxy_result
print(json.dumps(xproxy_result, indent=4))

this is a request tag test
{
    "cost": {
        "currency": "usd",
        "input": {
            "base": 8e-06
        },
        "output": {
            "base": 9e-06
        },
        "total": {
            "base": 1.7e-05
        }
    },
    "request_id": "40000b88-0000-fe00-b63f-84710c7967bb",
    "request_tags": [
        "x",
        "y"
    ]
}


Create a budget and make a request with that budget

In [4]:
from payi import Payi

payi_client = Payi(
    payi_api_key=payi_api_key,
    base_url=payi_base_url
)

#Create a budget
budget_response = payi_client.budgets.create(
    #Use a random name for the budget
    budget_name=''.join(random.choices(string.ascii_letters + string.digits, k=6)), 
    max=12.50, #$12.50 USD
    budget_type="Liberal",
    budget_response_type="Allow",
    budget_tags=["example_budget"]
)

budget_name = budget_response.budget.budget_name
budget_id = budget_response.budget.budget_id

print("Budget Created")
print(f"Budget Name: {budget_name}")
print(f"Budget ID: {budget_id}")

#Make a request using the new budget and request tags
response = oai_client.chat.completions.create(
    model="gpt-3.5-turbo", 
    messages=[{"role": "user", "content": "Say 'this is a budget test'"}],
    extra_headers=create_headers(
        request_tags=["x", "y"], 
        budget_ids=[budget_id]
    )
)

completion = response.choices[0].message.content
print(completion)

xproxy_result = response.xproxy_result
print(json.dumps(xproxy_result, indent=4))

Budget Created
Budget Name: NPpiEi
Budget ID: 4d5072a8-27fe-4188-94fd-08dc9f89e050
This is a budget test.
{
    "cost": {
        "currency": "usd",
        "input": {
            "base": 7.5e-06
        },
        "output": {
            "base": 9e-06
        },
        "total": {
            "base": 1.65e-05
        }
    },
    "request_id": "40000b8b-0000-fe00-b63f-84710c7967bb",
    "request_tags": [
        "x",
        "y"
    ],
    "budgets": {
        "4d5072a8-27fe-4188-94fd-08dc9f89e050": {
            "state": "ok"
        }
    }
}


See budget status

In [5]:
response = payi_client.budgets.retrieve(budget_id=budget_id)
print(f"Budget Name: {response.budget.budget_name}")
print(f"Budget ID: {response.budget.budget_id}")
print(f"Budget Creation Timestamp: {response.budget.budget_creation_timestamp}")
print(f"Budget Tags: {response.budget.budget_tags}")
print(f"Budget Input Base Cost: {response.budget.totals.cost.input.base}")
print(f"Budget Output Base Cost: {response.budget.totals.cost.output.base}")
print(f"Budget Total Base Cost: {response.budget.totals.cost.output.base}")

Budget Name: NPpiEi
Budget ID: 4d5072a8-27fe-4188-94fd-08dc9f89e050
Budget Creation Timestamp: 2024-07-08 23:47:37.909364
Budget Tags: ['example_budget']
Budget Input Base Cost: 7.5e-06
Budget Output Base Cost: 9e-06
Budget Total Base Cost: 9e-06


Make an ingest call with pre-computed token values

In [6]:
response = payi_client.ingest.units(
    category="system.openai",
    resource="gpt-4o",
    input=50,
    output=100,
    budget_ids= [budget_id],
    request_tags = ["a", "b"]
)

print(f"Ingest request ID: {response.request_id}")
print(f"Input Base Cost: {response.cost.input.base}")
print(f"Output Base Cost: {response.cost.output.base}")
print(f"Total Base Cost: {response.cost.total.base}")

Ingest request ID: 40000b8e-0000-fe00-b63f-84710c7967bb
Input Base Cost: 0.00025
Output Base Cost: 0.0015
Total Base Cost: 0.00175


Reset a budget back to zero tracked cost

In [ ]:
response = payi_client.budgets.reset(budget_id=budget_id)
print(response.message)
print("State prior to reset: ")
print(f"Budget Name: {response.budget_history.budget_name}")
print(f"Budget ID: {response.budget_history.budget_id}")
print(f"Budget Tags: {response.budget_history.budget_tags}")
print(f"Budget Reset Timestamp: {response.budget_history.budget_reset_timestamp}")
print(f"Budget Input Base Cost: {response.budget_history.totals.cost.input.base}")
print(f"Budget Output Base Cost: {response.budget_history.totals.cost.output.base}")
print(f"Budget Total Base Cost: {response.budget_history.totals.cost.total.base}")

print("\nState after reset:")
response = payi_client.budgets.retrieve(budget_id=budget_id)
print(f"Budget Name: {response.budget.budget_name}")
print(f"Budget ID: {response.budget.budget_id}")
print(f"Budget Creation Timestamp: {response.budget.budget_creation_timestamp}")
print(f"Budget Tags: {response.budget.budget_tags}")
print(f"Budget Input Base Cost: {response.budget.totals.cost}")
print(f"Budget Output Base Cost: {response.budget.totals.cost.output.base}")
print(f"Budget Total Base Cost: {response.budget.totals.cost.total.base}")

Create a small, conservative budget that will prevent calls from happening that exceed the maximum, then capture the output.

In [7]:
budget_response = payi_client.budgets.create(
    budget_name=''.join(random.choices(string.ascii_letters + string.digits, k=6)),  
    max=0.01, 
    budget_type="conservative",
    budget_response_type="block",
    budget_tags=["budget_block_example"]
)
cons_budget = budget_response.budget.budget_id

print("Budget Created")
print(f"Budget Name: {budget_response.budget.budget_name}")
print(f"Budget ID: {budget_response.budget.budget_id}")

try:
    response = oai_client.chat.completions.create(
        model="gpt-4-0125-preview", 
        messages=[{"role": "user", "content": "tell me a short story about a toy"}],
        extra_headers=create_headers(
            request_tags=["toy_story"], 
            budget_ids=[cons_budget]
        )
    )
except Exception as e:
    print(json.dumps(e.body, indent=4))

Budget Created
Budget Name: 6xqb3Q
Budget ID: fc8058bc-6922-4b16-94fe-08dc9f89e050
{
    "xproxy_result": {
        "blocked_budget_ids": [
            "fc8058bc-6922-4b16-94fe-08dc9f89e050"
        ],
        "request_id": "400018c9-0000-f400-b63f-84710c7967bb",
        "request_tags": [
            "toy_story"
        ],
        "budgets": {
            "fc8058bc-6922-4b16-94fe-08dc9f89e050": {
                "state": "blocked"
            }
        }
    },
    "message": "Insufficient budget for the requested operation",
    "statusCode": 400,
    "xproxy_error": {
        "code": "not_enough_budget",
        "message": "Insufficient budget for the requested operation"
    }
}


List and then delete all budgets

In [8]:
response = payi_client.budgets.list()
for budget in response.items:
    print("Deleting budget with id:" + budget.budget_id)
    payi_client.budgets.delete(budget.budget_id)

Deleting budget with id:4d5072a8-27fe-4188-94fd-08dc9f89e050
Deleting budget with id:fc8058bc-6922-4b16-94fe-08dc9f89e050
